# CS 159 Project

### 1. Imports

In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

import torch 

import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

print("All packages imported!")

### 2. Data

#### Download

In [ ]:
# credentials
%load_ext aicrowd.magic
%aicrowd login

In [ ]:
aicrowd_challenge_name = "mabe-2022-mouse-triplets"
if not os.path.exists('data'):
    os.mkdir('data')
    
if not os.path.exists('data/mabe'):
    os.mkdir('data/mabe')

# %aicrowd ds dl -c {aicrowd_challenge_name} -o data/mabe # Download all files
%aicrowd ds dl -c {aicrowd_challenge_name} -o data/mabe *submission_data* # download only the submission keypoint data
%aicrowd ds dl -c {aicrowd_challenge_name} -o data/mabe *user_train* # download data with the public task labels provided

#### Processing